# Prepare Dataset for Usage in CityLearn from Pre-Processed Data CSVs (from sub-dirs)

Technical note: in the data preparation process, all datetime stamps are given in `%Y-%m-%d %H:%M:%S` format, and all variables are given as the value of the quantity over the period (hour) beginning at the timestamp. Note that CityLearn uses the hour 24 convention (24th hour of previous day), but Python standard timestamps use hour 0. So once data has been collected, it is then converted to hour 24 format for savings as a CityLearn compatible dataset (though note only the building CSVs contain time information).

In [30]:
import pandas as pd
import numpy as np
import datetime
import os
import csv
import json
import itertools

## Configure dataset to be constructed

**NOTE! Remember to check asset capacity overwriting cell**

In [31]:
# set dataset info
dataset_name = 'v2-test'
vnum = '2-0' # version number
dataset_notes = '...'

elec_only = False
battery_efficiency = 0.9

# specify buildings to include in dataset - by ID
building_ids = [0,2,120]

# specify years of data to include in dataset
metering_years = [2019,2020,2022]
pricing_years = [2021] # available: 2019 to 2023
carbon_years = [2021] # available: 2009 to 2023
# NOTE: pricing_years and carbon_years are extended by repetition to match the length of metering_years

---

In [32]:
metering_years = sorted(metering_years)
pricing_years = sorted(pricing_years)
carbon_years = sorted(carbon_years)
dt_stamps = [ts for y in metering_years for ts in pd.date_range(start=pd.Timestamp(day=1,month=1,year=y,hour=0), end=pd.Timestamp(day=31,month=12,year=y,hour=23), freq='h')]

In [33]:
# specify path to data files to use
data_sources = {
    'cebd_path_pattern': os.path.join('building_data','processed_data','UCam_Building_b%s'),
    'building_ids': [f'b{n}' for n in building_ids],
    'floor_roof_areas': os.path.join('building_data','processed_data','building_floor_roof_areas.csv'),
    'weather_path': os.path.join('aux_data','MetOffice Weather Data','processed_data','bedford'),
    'solar_path': os.path.join('aux_data','RenewablesNinja Generation Data','processed_data','cambridge_52-194_0-131'),
    'pricing_path': os.path.join('aux_data','AgileOctopus Pricing Data','processed_data','import_price_regA'),
    'carbon_intensity_path': os.path.join('aux_data','NationalGridESO Carbon Intensity Data','processed_data')
}

# specify metadata for dataset
metadata = {
    # !!! edit this
    'name': dataset_name,
    'version': vnum,
    'metering_data_years': metering_years,
    'pricing_data_years': pricing_years,
    'carbon_data_years': carbon_years,
    'timezone': 'Europe/London', # from valid pytz.all_timezones
    'time_created': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'notes': dataset_notes,
    'data_source_info': data_sources
}
metadata.update({'save_dir': os.path.join('datasets','{0}-v{1}'.format(metadata['name'],metadata['version']))})

---

## Prepare data

In [34]:
# check building load data is available for requested years
for year in metering_years:
    for b_id in building_ids:
        assert os.path.exists(os.path.join(data_sources['cebd_path_pattern']%b_id,'electricity',f'{year}.csv')), 'No eletricity data for building b%d in year %d' % (b_id,year)
        if not elec_only:
            assert os.path.exists(os.path.join(data_sources['cebd_path_pattern']%b_id,'gas',f'{year}.csv')), 'No gas data for building b%d in year %d' % (b_id,year)

In [35]:
# check if weather data available for all metering years
unav_weather_years  = []
for year in metering_years:
    if not os.path.exists(os.path.join(data_sources['weather_path'],f'{year}.csv')):
        unav_weather_years.append(year)

if len(unav_weather_years) > 0:
    print(f'WARNING: Weather data not available for years: {unav_weather_years}')

In [36]:
# load in data
elec_load_dfs = [pd.concat([
    pd.read_csv(os.path.join(data_sources['cebd_path_pattern']%b_id,'electricity',f'{year}.csv')) for year in metering_years
    ], ignore_index=True) for b_id in building_ids]
if not elec_only:
    heat_load_dfs = [pd.concat([
        pd.read_csv(os.path.join(data_sources['cebd_path_pattern']%b_id,'gas',f'{year}.csv')) for year in metering_years
        ], ignore_index=True) for b_id in building_ids]

weather_df = pd.concat([
    pd.read_csv(os.path.join(data_sources['weather_path'],f'{year}.csv'), usecols=['datetime','air_temperature [degC]','rltv_hum [%]']
                ) for year in metering_years if year not in unav_weather_years], ignore_index=True)
null_weather_dfs = [pd.DataFrame({
    'datetime': pd.date_range(
        start=pd.Timestamp(day=1,month=1,year=year,hour=0),
        end=pd.Timestamp(day=31,month=12,year=year,hour=23), freq='h'
        ),
    'air_temperature [degC]': 20, # placeholder temp
    'rltv_hum [%]': 0 # placeholder humidity
    }) for year in unav_weather_years]
weather_df = pd.concat([weather_df,*null_weather_dfs], ignore_index=True)

solar_df = pd.concat([
    pd.read_csv(
        os.path.join(data_sources['solar_path'],f'{year}.csv'),
        usecols=['datetime','solar generation [W/kW]','irradiance_direct [W/m2]','irradiance_diffuse [W/m2]']
        ) for year in metering_years], ignore_index=True)

In [37]:
def fit_df_to_year(df, target_year, variable):
    """Adjust year of df timestamps and account for leap years by removing/repeating data."""

    df['datetime'] = pd.to_datetime(df['datetime'])
    current_year = df['datetime'].dt.year.unique()[0]

    if (not pd.Timestamp(year=current_year,month=1,day=1).is_leap_year) and (pd.Timestamp(year=target_year,month=1,day=1).is_leap_year):
        # repeat Feb28 data to Feb29
        feb28_values = df[(df['datetime'].dt.month==2) & (df['datetime'].dt.day==28)][variable].to_list()
        feb29_timestamps = pd.date_range(start=pd.Timestamp(day=29,month=2,year=target_year,hour=0), end=pd.Timestamp(day=29,month=2,year=target_year,hour=23), freq='h')
        df['datetime'] = df['datetime'].apply(lambda x: x.replace(year=target_year)) # change year to match target year to allow merge
        df = pd.concat([df,pd.DataFrame({'datetime': feb29_timestamps,variable: feb28_values})], ignore_index=True)

    elif (pd.Timestamp(year=current_year,month=1,day=1).is_leap_year) and (not pd.Timestamp(year=target_year,month=1,day=1).is_leap_year):
        # remove Feb29 pricing data
        df = df.drop(df[(df['datetime'].dt.month==2) & (df['datetime'].dt.day==29)].index)
        df['datetime'] = df['datetime'].apply(lambda x: x.replace(year=target_year)) # change year to match target year
    else:
        df['datetime'] = df['datetime'].apply(lambda x: x.replace(year=target_year)) # change year to match target year

    df.sort_values(by='datetime', inplace=True, ignore_index=True)

    return df

In [38]:
# extend pricing and carbon data, matching leap years
extended_price_years = list(itertools.islice(itertools.cycle(pricing_years),len(metering_years)))
pricing_year_dfs = []
for pyear,myear in zip(extended_price_years,metering_years):
    pyear_df = pd.read_csv(os.path.join(data_sources['pricing_path'],f'{pyear}.csv'))
    pricing_year_dfs.append(fit_df_to_year(pyear_df,myear,'Electricity Pricing [£/kWh]'))
pricing_df = pd.concat(pricing_year_dfs, ignore_index=True)
pricing_df['Electricity Pricing [£/kWh]'] = pricing_df['Electricity Pricing [£/kWh]'].clip(0) # clip prices at zero

extended_carbon_years = list(itertools.islice(itertools.cycle(carbon_years),len(metering_years)))
carbon_year_dfs = []
for cyear,myear in zip(extended_carbon_years,metering_years):
    cyear_df = pd.read_csv(os.path.join(data_sources['carbon_intensity_path'],f'{cyear}.csv'))
    carbon_year_dfs.append(fit_df_to_year(cyear_df,myear,'Carbon Intensity [kg_CO2/kWh]'))
carbon_df = pd.concat(carbon_year_dfs, ignore_index=True)

In [39]:
# check all dfs have required length
all_dfs = [*elec_load_dfs,*heat_load_dfs,weather_df,solar_df,pricing_df,carbon_df]
df_names = [*[f'elec_b{b_id}' for b_id in building_ids],*[f'heat_b{b_id}' for b_id in building_ids],'weather','solar','pricing','carbon']

for df in all_dfs:
    df['datetime'] = pd.to_datetime(df['datetime'])

for df,name in zip(all_dfs,df_names):
    assert len(df) == len(dt_stamps), f'Dataframe length mismatch for df {name}, {len(df)} != {len(dt_stamps)}'
# they should all have the same timestamps as well
for df,name in zip(all_dfs,df_names):
    assert (df['datetime'] == dt_stamps).all(), f'Timestamps do not match for df {name}'

## Combine data sources into required files (dataframes)

In [40]:
# construct time information for buildings
times_df = pd.DataFrame(data=[pd.Timestamp(ts,tz='UTC') for ts in dt_stamps], columns=['timestamp'])
times_df['month'] = times_df['timestamp'].dt.month
times_df['day of week'] = times_df['timestamp'].dt.dayofweek
times_df['hour'] = times_df['timestamp'].dt.hour
times_df['daysave'] = [1 if (ts.astimezone(tz=metadata['timezone']).dst() == pd.Timedelta(1,'h')) else 0 for ts in times_df['timestamp'].to_list()]
# convert to EnergyPlus day numbering convention - Sun -> 1, ... , Sat -> 7
# https://bigladdersoftware.com/epx/docs/8-1/input-output-reference/page-009.html
# note, I will ignore 'special' days (type 8)
times_df['day of week'] += 2
times_df['day of week'] = times_df['day of week'].replace(8,1)
# convert to hour 24 format - i.e. midnight is 24th hour of previous day
times_df['hour'] = times_df['hour'].replace(0,24)
times_df.loc[times_df['hour'] == 24, 'day of week'] -= 1
times_df['day of week'] = times_df['day of week'].replace(0,7)
times_df.loc[[(ts.is_month_start and hr == 24) for ts,hr in zip(times_df['timestamp'],times_df['hour'])], 'month'] -= 1
times_df['month'] = times_df['month'].replace(0,12)

In [41]:
building_files = []
for elec_df,heat_df,id in zip(elec_load_dfs,heat_load_dfs,data_sources['building_ids']):
    building_file = times_df[['month','hour','day of week','daysave']].copy()
    building_file.rename(columns={'month':'Month','hour':'Hour','day of week':'Day Type','daysave':'Daylight Savings Status'},inplace=True)
    building_file['Indoor Temperature [C]'] = ""
    building_file['Average Unmet Cooling Setpoint Difference [C]'] = ""
    building_file['Indoor Relative Humidity [%]'] = ""
    building_file['Equipment Electric Power [kWh]'] = elec_df['equipment load [kWh]']
    building_file['DHW Heating [kWh]'] = 0
    building_file['Cooling Load [kWh]'] = 0
    building_file['Heating Load [kWh]'] = 0 if elec_only else heat_df['heating load [kWh]']
    building_file['Solar Generation [W/kW]'] = solar_df['solar generation [W/kW]']

    building_files.append(building_file)

In [42]:
carbon_intensity_file = pd.DataFrame(data=carbon_df['Carbon Intensity [kg_CO2/kWh]'].to_list(),columns=['kg_CO2/kWh'])

In [43]:
pricing_file = pd.DataFrame(data=pricing_df['Electricity Pricing [£/kWh]'].to_list(),columns=['Electricity Pricing [£/kWh]'])
for dt in [6,12,24]:
    pricing_file['%sh Prediction Electricity Pricing [£/kWh]'%dt] = pricing_file['Electricity Pricing [£/kWh]'].shift(periods=-dt)

In [44]:
weather_cols = ['Outdoor Drybulb Temperature [C]',
    'Relative Humidity [%]',
    'Diffuse Solar Radiation [W/m2]',
    'Direct Solar Radiation [W/m2]']

weather_file = pd.DataFrame(data=np.array([
    weather_df['air_temperature [degC]'].to_list(),
    weather_df['rltv_hum [%]'].to_list(),
    solar_df['irradiance_diffuse [W/m2]'].to_list(),
    solar_df['irradiance_direct [W/m2]'].to_list()
    ]).T,
    columns=weather_cols)

for col_name in weather_cols:
    for dt in [6,12,24]:
        weather_file[('%sh Prediction '%dt + col_name)] = weather_file[col_name].shift(periods=-dt)

## Set up data writing variables

In [45]:
building_fname_pattern = 'UCam_Building_b%s.csv'
carbon_fname = 'carbon_intensity.csv'
pricing_fname = 'pricing.csv'
weather_fname = 'weather.csv'
timedata_fname = 'timestamps.csv'
schema_fname = 'schema.json'
mdata_fname = 'metadata.json'

## Create `schema.json`

In [46]:
# get building area attributes
floor_roof_areas_df = pd.read_csv(data_sources['floor_roof_areas'],usecols=['Building ID','GIA (m2)','Number of floors'])

floor_areas = {}
num_floors = {}
approx_roof_areas = {}
for b_id in data_sources['building_ids']:
    floor_areas[b_id] = np.round(*floor_roof_areas_df[floor_roof_areas_df['Building ID'] == b_id]['GIA (m2)'].values,1)
    num_floors[b_id] = float(floor_roof_areas_df[floor_roof_areas_df['Building ID'] == b_id]['Number of floors'].values[0])
    approx_roof_areas[b_id] = np.round(floor_areas[b_id]/num_floors[b_id],1)

floor_areas,num_floors,approx_roof_areas = [dict(sorted(d.items(), key=lambda item: item[0])) for d in (floor_areas,num_floors,approx_roof_areas)]

In [47]:
# optional pre-processing to decide how to size the batteries & pv panels

storage_hours = 24 # no. of hours of mean load that can be stored in battery
storage_power_factor = 3 # proportion of mean load that battery can provide as output power
#solar_power_factor = 1 # proportion of mean load that pv panel can provide as peak power
panel_fill_factor = 0.9 # proportion of roof filled by PV panels
panel_power_density = 0.15 # (kWp/m2) panel peak (nominal) power per area
# panel refs: https://www.sciencedirect.com/science/article/pii/S0378778822002547 (Table 1), https://iopscience.iop.org/article/10.1088/1748-9326/aaa554/meta (Table 1), https://www.pvfitcalculator.energysavingtrust.org.uk/Documents/150224_SolarEnergy_Calculator_Sizing_Guide_v1.pdf

In [48]:
# set schema building parameters
battery_efficiencies = {id: battery_efficiency for id in sorted(data_sources['building_ids'])}

id_file_zip = sorted(list(zip(building_files,data_sources['building_ids'])), key=lambda x: x[1])
mean_loads = {id: np.round(np.mean(building_file['Equipment Electric Power [kWh]']),1) for building_file,id in id_file_zip}
battery_energy_capacities = {id: np.round(storage_hours*mean_loads[id],0) for id in sorted(data_sources['building_ids'])}
battery_power_capacities = {id: np.round(storage_power_factor*mean_loads[id],0) for id in sorted(data_sources['building_ids'])}
#pv_power_capacities = {id: np.round(solar_power_factor*mean_loads[id],0) for id in sorted(data_sources['building_ids'])}
pv_power_capacities = {id: np.round(approx_roof_areas[id]*panel_fill_factor*panel_power_density,0) for id in sorted(data_sources['building_ids'])}

print(mean_loads)
print(battery_energy_capacities)
print(battery_power_capacities)
print(pv_power_capacities)

{'b0': 22.4, 'b120': 11.4, 'b2': 871.4}
{'b0': 538.0, 'b120': 274.0, 'b2': 20914.0}
{'b0': 67.0, 'b120': 34.0, 'b2': 2614.0}
{'b0': 89.0, 'b120': 162.0, 'b2': 342.0}


In [49]:
# override building parameters for consistency between train, test, validate sets (set using train data)
...

In [50]:
building_mdata = {
    'mean_loads (kW)': mean_loads,
    'battery_efficiencies (-)': battery_efficiencies,
    'battery_energy_capacities (kWh)': battery_energy_capacities,
    'battery_power_capacities (kW)': battery_power_capacities,
    'pv_power_capacities (kW)': pv_power_capacities,
    'gross_internal_floor_area (m2)': floor_areas,
    'number_of_floors (-)': num_floors,
    'approx_roof_area (m2)': approx_roof_areas
}
metadata.update({'building_attributes': building_mdata})

In [51]:
# load base schema
with open(os.path.join('resources','base_schema.json')) as base_schema:
    schema = json.load(base_schema)

schema['simulation_end_time_step'] = len(dt_stamps) - 1 # set length of simulation

# write building attributes
schema['buildings'] = {}
for id in sorted(data_sources['building_ids']):

    int_id = id.replace('b','')

    building_dict = {
        'include': True,
        'energy_simulation': building_fname_pattern%int_id,
        'weather': weather_fname,
        'carbon_intensity': carbon_fname,
        'pricing': pricing_fname,
        'inactive_observations': [],
        'inactive_actions': [],

        'electrical_storage': {
            'type': "citylearn.energy_model.Battery",
            'autosize': False,
            'attributes': {
                'capacity': battery_energy_capacities[id],
                'nominal_power': battery_power_capacities[id],
                'capacity_loss_coefficient': 1e-05,
                'loss_coefficient': 0,
                'power_efficiency_curve': [[0,battery_efficiencies[id]],[1,battery_efficiencies[id]]],
                'capacity_power_curve': [[0,1],[1,1]]
            }
        },

        'pv': {
            'type': "citylearn.energy_model.PV",
            'autosize': False,
            'attributes': {
                'nominal_power': pv_power_capacities[id]
            }
        }
    }

    schema['buildings'].update({building_fname_pattern%int_id: building_dict})

## Save files and metadata

In [52]:
print(metadata)

{'name': 'v2-test', 'version': '2-0', 'metering_data_years': [2019, 2020, 2022], 'pricing_data_years': [2021], 'carbon_data_years': [2021], 'timezone': 'Europe/London', 'time_created': '2024-04-09 14:31:44', 'notes': '...', 'data_source_info': {'cebd_path_pattern': 'building_data/processed_data/UCam_Building_b%s', 'building_ids': ['b0', 'b2', 'b120'], 'floor_roof_areas': 'building_data/processed_data/building_floor_roof_areas.csv', 'weather_path': 'aux_data/Met Office Weather Data/processed_data/bedford', 'solar_path': 'aux_data/RenewablesNinja Generation Data/processed_data/cambridge_52-194_0-131', 'pricing_path': 'aux_data/AgileOctopus Pricing Data/processed_data/import_price_regA', 'carbon_intensity_path': 'aux_data/NationalGridESO Carbon Intensity Data/processed_data'}, 'save_dir': 'datasets/v2-test-v2-0', 'building_attributes': {'mean_loads (kW)': {'b0': 22.4, 'b120': 11.4, 'b2': 871.4}, 'battery_efficiencies (-)': {'b0': 0.9, 'b120': 0.9, 'b2': 0.9}, 'battery_energy_capacities (k

In [53]:
# save dataframes & metadata
if not os.path.exists(metadata['save_dir']):
    os.mkdir(metadata['save_dir'])

for building_file,id in  zip(building_files,building_ids):
    building_file.to_csv(os.path.join(metadata['save_dir'],building_fname_pattern%id), index=False)

carbon_intensity_file.to_csv(os.path.join(metadata['save_dir'],carbon_fname), index=False)
pricing_file.to_csv(os.path.join(metadata['save_dir'],pricing_fname), index=False)
weather_file.to_csv(os.path.join(metadata['save_dir'],weather_fname), index=False)

with open(os.path.join(metadata['save_dir'],timedata_fname), 'w') as file:
    writer = csv.writer(file)
    writer.writerow(['Timestamp (UTC)'])
    writer.writerows([[tstamp] for tstamp in dt_stamps])

with open(os.path.join(metadata['save_dir'],schema_fname), 'w') as file:
    json.dump(schema, file, indent=4)

with open(os.path.join(metadata['save_dir'],mdata_fname), 'w') as file:
    json.dump(metadata, file, indent=4)